- load submission with iterative imp and reconstruct dataset with it - DONE

- for col in cols with missing values (or only F2_x and F4_x ?)
    - col is target
    - other cols are features
    - complete rows are for training
    - incomplete (with missing value) are for prediction
    - when the prediction is complete, update column values in dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

# Load Data

In [2]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col = 'row-col')
sub_iter = pd.read_csv('../input/submission-iterativeimp/submission_iterative-imp.csv', index_col = 'row-col')
#sub_iter = pd.read_csv('../input/submission-iterF4/submission_iterativeF_4.csv', index_col = 'row-col')

# display(data,submission,sub_iter)

In [3]:
data

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,...,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733
999996,-0.769106,-0.387363,-1.227469,0.601183,0.351161,0.219475,-0.530277,0.853452,0.608646,1.648023,...,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807
999997,0.147534,-0.715276,-0.465049,-1.988941,-1.594535,-1.044882,3.159455,-0.634108,0.890382,-1.212444,...,2.891854,3.105002,-3.470520,NaN,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502


In [4]:
# # discussion says that mean for F_1 & 3 is better than everything else. Let's try.
# from sklearn.impute import SimpleImputer

# data_imp = data.copy()

# imp = SimpleImputer(strategy='mean')
# data_imp[:] = imp.fit_transform(data_imp)

# data_imp

In [5]:
# # now use prepared predictions for F_4_*
# for i in tqdm(sub_iter.index):
#     row = int(i.split('-')[0])
#     col = i.split('-')[1]
#     if col.startswith('F_4_'):
#         data_imp.at[row,col] = sub_iter.loc[i]
        
# data_imp

# Apply results from Iterative Imputer

In [6]:
# use results from iterative imputation as a base for next steps -> fill values into data
data_imp = data.copy()
for i in tqdm(sub_iter.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    data_imp.at[row,col] = sub_iter.loc[i]

100%|██████████| 1000000/1000000 [01:54<00:00, 8744.69it/s]


# Treat cols with missing values as targets

Let's do this part only for columns F_4_*
others have probably no impact. 

https://www.kaggle.com/code/jadelsoufi/bayesian-optimization-xgboost-lgbm/comments?scriptVersionId=99389640

In [7]:
%%time
 
from xgboost import XGBRegressor

# https://www.kaggle.com/code/jadelsoufi/bayesian-optimization-xgboost-lgbm/notebook?scriptVersionId=99389640
params={'colsample_bytree': 0.8756732201834143,
        'eta': 0.08537397408966756,
        'gamma': 4.06563425414554,
        'max_depth': 7,
        'min_child_weight': 4}

model = XGBRegressor(**params)


# select only F4 columns
cols_F4 = [c for c in data.columns if c.startswith('F_4_')]


for col in tqdm(cols_F4):
    #print(f'column:{col}\n')
    
    # do in every cycle to update data with latest predictions
    data_F4 = data_imp[cols_F4]
    
    #display(data_F4)
    
    # prepare dframes
    train_df = data_F4[data[col].notna()]
    test_df = data_F4[data[col].isna()]
    
    X_train = train_df.copy()
    y_train = X_train.pop(col)
    X_test = test_df.drop(col,axis=1)
    
#     display(X_train.columns, X_test.columns)
    
#     display(train_df,test_df)
#     display(X_train,y_train,X_test)
#     display(X_train,y_train,X_test)
    
    # model: train + predict
    model.fit(X_train,y_train)
    y = model.predict(X_test)
    
    # update df with new predictions
    data_imp.loc[data[col].isna(),col] = y
    

100%|██████████| 15/15 [50:37<00:00, 202.52s/it]

CPU times: user 3h 18min 11s, sys: 14.3 s, total: 3h 18min 25s
Wall time: 50min 39s


In [8]:
# now plug the predictions into original dataset and use mean for all variables which are still missing.
from sklearn.impute import SimpleImputer

data[cols_F4] = data_imp[cols_F4]

# mean
imp = SimpleImputer(strategy='mean')

data[:] = imp.fit_transform(data)


# Submission

In [9]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i,'value'] = data.loc[row,col]
    
    
submission.to_csv('submission.csv')

100%|██████████| 1000000/1000000 [01:46<00:00, 9410.15it/s]
